# Crawl4AI with local LLMs

In [ ]:
# Linux installation (run on terminal)
# curl -fsSL https://ollama.com/install.sh | sh

# Verify
# ollama -v



In [6]:
# Install the package
!pip install -U crawl4ai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━ 431.3/431.3 KB 94.4 kB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 63.0 kB/s eta 0:00:00m eta 0:00:010:00:26
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 64.4 kB/s eta 0:00:00m eta 0:00:010:00:06
     ━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 KB 58.4 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 57.0 kB/s eta 0:00:00m eta 0:00:010:00:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 KB 49.9 kB/s eta 0:00:00 kB/s eta 0:00:01:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 KB 69.7 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 63.5 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.4 kB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 KB 51.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 KB

In [ ]:
# python -m playwright install --with-deps chromium

Defaulting to user installation because normal site-packages is not writeable


In [11]:
!crawl4ai-doctor

[INIT].... → Running Crawl4AI health check... 
[INIT].... → Crawl4AI 0.7.6 
[TEST].... ℹ Testing crawling capabilities... 
[EXPORT].. ℹ Exporting media (PDF/MHTML/screenshot) took 0.65s 
[FETCH]... ↓ https://crawl4ai.com                                  
| ✓ | ⏱: 4.07s 
[SCRAPE].. ◆ https://crawl4ai.com                                  
| ✓ | ⏱: 0.02s 
[COMPLETE] ● https://crawl4ai.com                                  
| ✓ | ⏱: 4.10s 
[COMPLETE] ● ✅ Crawling test passed! 


# Basic web crawling

Provided in the docs:

In [14]:
# If running in notebooks
import nest_asyncio
nest_asyncio.apply()

In [15]:
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode

async def main():
    browser_conf = BrowserConfig(headless=True)  # or False to see the browser
    run_conf = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS
    )

    async with AsyncWebCrawler(config=browser_conf) as crawler:
        result = await crawler.arun(
            url="https://example.com",
            config=run_conf
        )
        print(result.markdown)

if __name__ == "__main__":
    asyncio.run(main())

[INIT].... → Crawl4AI 0.7.6 
[FETCH]... ↓ https://example.com                                   
| ✓ | ⏱: 0.75s 
[SCRAPE].. ◆ https://example.com                                   
| ✓ | ⏱: 0.00s 
[COMPLETE] ● https://example.com                                   
| ✓ | ⏱: 0.75s 
# Example Domain
This domain is for use in documentation examples without needing permission. Avoid use in operations.
[Learn more](https://iana.org/domains/example)



## Use with local LLM

In [ ]:
# download your model in terminal
# ollama pull llama3.2:1b

In [17]:
import os
import asyncio
import json
from pydantic import BaseModel, Field
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode, LLMConfig
from crawl4ai import LLMExtractionStrategy

class Product(BaseModel):
    name: str
    price: str

async def main():
    # 1. Define the LLM extraction strategy
    llm_strategy = LLMExtractionStrategy(
        llm_config = LLMConfig(provider="ollama/llama3.2:1b", api_token=None),
        schema=Product.schema_json(), # Or use model_json_schema()
        extraction_type="schema",
        instruction="Extract all the titles of each article in the website.",
        chunk_token_threshold=1000,
        overlap_rate=0.0,
        apply_chunking=True,
        input_format="markdown",   # or "html", "fit_markdown"
        extra_args={"temperature": 0.0, "max_tokens": 800}
    )

    # 2. Build the crawler config
    crawl_config = CrawlerRunConfig(
        extraction_strategy=llm_strategy,
        cache_mode=CacheMode.BYPASS
    )

    # 3. Create a browser config if needed
    browser_cfg = BrowserConfig(headless=True)

    async with AsyncWebCrawler(config=browser_cfg) as crawler:
        # 4. Let's say we want to crawl a single page
        result = await crawler.arun(
            url="https://www.scrapethissite.com/pages/",
            config=crawl_config
        )

        if result.success:
            # 5. The extracted content is presumably JSON
            data = json.loads(result.extracted_content)
            print("Extracted items:", data)

            # 6. Show usage stats
            llm_strategy.show_usage()  # prints token usage
        else:
            print("Error:", result.error_message)

if __name__ == "__main__":
    asyncio.run(main())


<positron-console-cell-17>:17: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/


[INIT].... → Crawl4AI 0.7.6 
[FETCH]... ↓ https://www.scrapethissite.com/pages/                 
| ✓ | ⏱: 0.86s 
[SCRAPE].. ◆ https://www.scrapethissite.com/pages/                 
| ✓ | ⏱: 0.01s 
[EXTRACT]. ■ https://www.scrapethissite.com/pages/                 
| ✓ | ⏱: 5.49s 
[COMPLETE] ● https://www.scrapethissite.com/pages/                 
| ✓ | ⏱: 6.36s 
Extracted items: [{'name': 'Countries of the World: A Simple Example', 'price': None, 'title': 'Name'}, {'name': 'Hockey Teams: Forms, Searching and Pagination', 'price': None, 'title': 'Price'}, {'name': 'Oscar Winning Films: AJAX and Javascript', 'price': None, 'title': 'Name'}, {'name': 'Turtles All the Way Down: Frames & iFrames', 'price': None, 'title': 'Price'}, {'name': "Advanced Topics: Real World Challenges You'll Encounter", 'price': None, 'title': 'Name'}, {'index': 0, 'error': True, 'tags': ['error'], 'content': ['{\\"properties\\": {\\"name\\": {\\"title\\": \\"Name\\", \\"type\\": \\"string\\"}, \\"price\\": {\\"t